In [1]:
import requests

1. Get class data

In [18]:
# course_name = "CIS-121"
# api_endpoint = f"v1/courses/{course_name}/reviews"
# token = "public"

# base_url = "http://api.penncoursereview.com/"
# query_url = f"{base_url}{api_endpoint}?token={token}"

headers = { 'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36', }

response = requests.get("https://penncoursereview.com/api/review/course/CIS-121?format=json&token=platform", headers=headers)


if response.status_code == 200:
    data = response.json()
    print(data)
else:
    print(f"Request failed with status code: {response.status_code}")


Request failed with status code: 403
